In [1]:
import time
import os
import sys
import numpy as np
#import ipympl
import matplotlib
#matplotlib.use('nbagg')
#from matplotlib import style
#style.use('ggplot')
import matplotlib.pyplot as plt

import astropy.units as u
from astropy import stats
from astropy.io import fits
from mmtwfs.wfs import *
from mmtwfs.zernike import ZernikeVector
from mmtwfs.telescope import MMT

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [3]:
def wfsrun(wfs, mode, file):
    r = wfs.measure_slopes(file, mode=mode, plot=True)
    return r

# Binospec Dev Section

In [4]:
bino = WFSFactory(wfs="binospec")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
fig, ax = plt.subplots()
ax.imshow(bino.pupil_mask(hdr=None))
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Header information not available for Binospec pupil mask. Assuming default position.


In [12]:
#%%prun
bino_file = "/Volumes/LaCie 8TB/wfsdat/20180208/wfs_ff_cal_img_2018.0208.074052.fits"
bino_file = "/Users/tim/MMT/mmtwfs/mmtwfs/data/test_data/wfs_ff_cal_img_2017.1113.111402.fits"
bino_file = "/Volumes/LaCie 8TB/wfsdat/20180209/wfs_ff_cal_img_2018.0209.114054.fits"
#bino_file = "/Volumes/LaCie 8TB/wfsdat/20180210/wfs_ff_cal_img_2018.0210.095719.fits"
#bino_file = "/Volumes/LaCie 8TB/wfsdat/20180210/wfs_ff_cal_img_2018.0210.102615.fits"
#bino_file = "/Volumes/LaCie 8TB/wfsdat/20180604/wfs_ff_cal_img_2018.0604.052401.fits"
#bino_file = "/Volumes/LaCie 8TB/wfsdat/20191119/wfs_ff_cal_img_2019.1119.031015.fits"
#bino_file = "/Volumes/LaCie 8TB/wfsdat/20200316/wfs_ff_cal_img_2020.0316.045519.fits"
#bino_file = "/Volumes/LaCie 8TB/wfsdat/20200316/wfs_ff_cal_img_2020.0316.121544.fits"
#bino_file = "/Volumes/LaCie 8TB/wfsdat/20200127/wfs_ff_cal_img_2020.0127.041552.fits"
#bino_file = "/Users/tim/MMT/mmtwfs/mmtwfs/data/ref_images/bino_mask.fits"
results = bino.measure_slopes(bino_file, mode="binospec", plot=True)
results['figures']['slopes'].show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
mask = mk_wfs_mask(results['data'][106:406,100:400], outfile="/Users/tim/MMT/mmtwfs/mmtwfs/data/ref_images/bino_mask.fits")

In [16]:
refaps = bino.modes['binospec']['reference'].masked_apertures
fig, ax = plt.subplots()
fit = results['grid_fit']
refx = refaps['xcentroid'] * (fit['scale'] + refaps['xcentroid'] * fit['xcoma']) + fit['xcen']
refy = refaps['ycentroid'] * (fit['scale'] + refaps['ycentroid'] * fit['ycoma']) + fit['ycen']
x, y = results['apertures'].positions.transpose()[0], results['apertures'].positions.transpose()[1]
spacing = 0.5*(results['xspacing'] + results['yspacing'])
ax.scatter(refx, refy)
ax.set_aspect('equal')
ax.scatter(x, y)
ax.set_xlim(0, 512)
ax.set_ylim(0, 512)
fig.show()

results['grid_fit']

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'xcen': 285.6303197996806,
 'ycen': 247.1240340653681,
 'scale': 0.9136045574746333,
 'xcoma': -4.201657883466427e-05,
 'ycoma': 7.209100626100136e-06}

In [18]:
fig, ax = plt.subplots()
ax.scatter(refaps['xcentroid'], refaps['ycentroid'])
ax.set_aspect('equal')
fig.show()
refaps['xcentroid'].max()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

149.17766808867196

In [22]:
# get rotation of reference LED image
x1, y1 = 4.5, 491.5
x2, y2 = 9.5, 10
(np.arctan2(5, 481.5) * u.radian).to(u.deg)

<Quantity 0.59495037 deg>

In [10]:
#%%prun
zresults = bino.fit_wavefront(results, plot=True)
zvec = zresults['zernike']
print(zresults['residual_rms'])
#zvec.normalize()
print(zvec.pretty_print())
#print(zvec)
zresults['resid_plot'].show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

954.6871185132911 nm
Fringe Coefficients
 Z02:         424.3 ±   163 nm 	 X Tilt (1, 1)
 Z03:         392.8 ±   169 nm 	 Y Tilt (1, -1)
 Z04:         -7208 ±  61.5 nm 	 Defocus (2, 0)
 Z05:         -2025 ±   111 nm 	 Primary Astig at 45° (2, -2)
 Z06:        -974.3 ±   118 nm 	 Primary Astig at 0° (2, 2)
 Z07:         835.1 ±  80.8 nm 	 Primary Y Coma (3, -1)
 Z08:        -256.3 ±  72.6 nm 	 Primary X Coma (3, 1)
 Z09:           214 ±   116 nm 	 Y Trefoil (3, -3)
 Z10:         134.6 ±  90.9 nm 	 X Trefoil (3, 3)
 Z11:        -97.83 ±  53.1 nm 	 Primary Spherical (4, 0)
 Z12:           296 ±  70.2 nm 	 Secondary Astigmatism at 0° (4, 2)
 Z13:         84.16 ±  72.3 nm 	 Secondary Astigmatism at 45° (4, -2)
 Z14:        -210.2 ±   122 nm 	 X Tetrafoil (4, 4)
 Z15:        -215.2 ±  83.4 nm 	 Y Tetrafoil (4, -4)
 Z16:        -373.6 ±  64.5 nm 	 Secondary X Coma (5, 1)
 Z17:        -174.3 ±  97.7 nm 	 Secondary Y Coma (5, -1)
 Z18:        -217.9 ±  61.4 nm 	 Secondary X Trefoil (5, 3)
 Z19: 

In [11]:
zvec.fringe_bar_chart().show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
zvec.bar_chart(last_mode=22).show()

In [ ]:
zvec.plot_map().show()

In [ ]:
zv = ZernikeVector(Z38=100)
ft, m1c, zv_masked = bino.telescope.calculate_primary_corrections(zvec, mask=['Z22'])
bino.telescope.to_rcell(ft, filename="null_forces")

In [ ]:
forces, m1focus, zv_masked = bino.calculate_primary(zvec, threshold=0.25*zresults['residual_rms'])

In [ ]:
bino.modes['binospec']['reference']

In [ ]:
bino.modes['binospec']['reference']['xspacing'], bino.modes['binospec']['reference']['yspacing']

In [ ]:
bino.modes['binospec']['reference']['figure'].show()

# MMIRS Dev Section

In [4]:
mmirs = WFSFactory(wfs="mmirs")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
plt.close('all')
#mmirs.cen_sigma = 6.0
mmirs_file = "/Users/tim/MMT/wfsdat/20181120/mmirs_wfs_%04d.fits" % 157
#mmirs_file = "/Volumes/LaCie 8TB/wfsdat/20181221/mmirs_wfs_%04d.fits" % 61
#mmirs_file = "/Volumes/LaCie 8TB/wfsdat/20171013/mmirs_wfs_rename_%04d.fits" % 566  # pacman = 565,566
#mmirs_file = "/Users/tim/MMT/mmtwfs/mmtwfs/data/test_data/mmirs_wfs_0150.fits"
#mmirs_file = "/Volumes/LaCie 8TB/wfsdat/20180511/mmirs_wfs_0275.fits"
#mmirs_file = "/Users/tim/MMT/wfsdat/test/mmirs_wfs_0192.fits"
#mmirs_file = "/Volumes/LaCie 8TB/wfsdat/20181121/mmirs_wfs_0009.fits"
#mmirs_file = "/Users/tim/MMT/wfsdat/20181120/mmirs_wfs_0125.fits"
#mmirs_file = "/Volumes/LaCie 8TB/wfsdat/20191213/mmirs_wfs_0039.fits"
#mmirs_file = "/Volumes/LaCie 8TB/wfsdat/20200109/mmirs_wfs_0135.fits"
#mmirs_file = "/Users/tim/MMT/mmtwfs/mmtwfs/data/test_data/mmirs_wfs_0150.fits"
results = mmirs.measure_slopes(mmirs_file, plot=True)
results['figures']['slopes'].show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# 107, 184  363 189
180. * np.arctan2(5, 256) / np.pi

In [ ]:
results['figures']['pupil_center'].show()

In [ ]:
results['grid_fit']

In [ ]:
mask = mk_wfs_mask(results['data'][75:395,85:425], outfile="/Users/tim/MMT/mmtwfs/mmtwfs/data/ref_images/mmirs_mask.fits")


In [ ]:
hdu = fits.PrimaryHDU(results['data'])
hdu.writeto('data.fits', overwrite=True)
mmirs.telescope.obscuration

In [27]:
mmirs.nzern = 10
zresults = mmirs.fit_wavefront(results, plot=True)
zvec = zresults['zernike']
print(zresults['residual_rms'])
#zvec.normalize()
print(zvec.pretty_print())
zresults['resid_plot'].show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1141.4053991164974 nm
Fringe Coefficients
 Z02:        -208.6 ±   137 nm 	 X Tilt (1, 1)
 Z03:         53.72 ±   156 nm 	 Y Tilt (1, -1)
 Z04:         360.5 ±  47.8 nm 	 Defocus (2, 0)
 Z05:         312.9 ±    89 nm 	 Primary Astig at 45° (2, -2)
 Z06:          40.8 ±   102 nm 	 Primary Astig at 0° (2, 2)
 Z07:        -188.6 ±  65.7 nm 	 Primary Y Coma (3, -1)
 Z08:        -188.2 ±  54.9 nm 	 Primary X Coma (3, 1)
 Z09:        -81.51 ±  83.2 nm 	 Y Trefoil (3, -3)
 Z10:         120.1 ±    84 nm 	 X Trefoil (3, 3)
 Z11:        -91.18 ±  36.3 nm 	 Primary Spherical (4, 0)

Total RMS: 	 270.3 nm



In [28]:
zresults['zernike'].fringe_bar_chart().show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
fig, ax = plt.subplots()
norm = wfs_norm(results['data'])
ax.imshow(results['data'], cmap='Greys', origin='lower', norm=norm, interpolation='None')
mmirs.modes['mmirs1']['reference'].photapers.plot(color='red', lw=1.5, alpha=0.1, ax=ax)
plt.show()

In [13]:
plt.close('all')

In [ ]:
refaps = mmirs.modes['mmirs1']['reference'].masked_apertures
fig, ax = plt.subplots()
fit = results['grid_fit']
refx = refaps['xcentroid'] * (fit['scale'] + refaps['xcentroid'] * fit['xcoma']) + fit['xcen']
refy = refaps['ycentroid'] * (fit['scale'] + refaps['ycentroid'] * fit['ycoma']) + fit['ycen']
x, y = results['apertures'].positions.transpose()[0], results['apertures'].positions.transpose()[1]
spacing = 0.5*(results['xspacing'] + results['yspacing'])
ax.scatter(refx, refy)
ax.set_aspect('equal')
ax.scatter(x, y)
ax.set_xlim(0, 512)
ax.set_ylim(0, 512)
fig.show()

results['grid_fit']

In [ ]:
zresults['zernike'].bar_chart(last_mode=21).show()

In [ ]:
psf, pfig = mmirs.telescope.psf(zresults['zernike'].copy())
pfig.show()

In [ ]:
print(mmirs.modes['mmirs1']['reference']['fwhm'])

spot = mmirs.modes['mmirs1']['reference']['spot']
model = Gaussian2D(amplitude=spot.max(), x_mean=spot.shape[1]/2, y_mean=spot.shape[0]/2) + Polynomial2D(degree=0)
#model = Gaussian2D(amplitude=spot.max(), x_mean=spot.shape[1]/2, y_mean=spot.shape[0]/2)
fitter = LevMarLSQFitter()
y, x = np.mgrid[:spot.shape[0], :spot.shape[1]]
fit = fitter(model, x, y, spot)
fig, ax = plt.subplots()
ax.imshow(mmirs.modes['mmirs1']['reference']['spot'] - fit(x, y))
fig.show()

In [ ]:
mmirs.modes['mmirs1']['reference']['figure'].show()
mmirs.modes['mmirs1']['reference']['xspacing'], mmirs.modes['mmirs1']['reference']['yspacing']

In [ ]:
zplot = zvec.copy()
zplot.ignore('Z02')
zplot.ignore('Z03')
mfig = zplot.plot_map()
mfig.show()
zplot.peak2valley

In [ ]:
zvec.units = u.nm
f = zvec.bar_chart(residual=zresults['residual_rms'])
f.set_size_inches(9, 5)

f.show()

In [ ]:
plt.close('all')

In [ ]:
torig = mmirs.telescope.bending_forces(zvec)
torig.show_in_notebook()

In [ ]:
force_fig = mmirs.telescope.plot_forces(torig)
force_fig.show()

In [ ]:
t, m1foc = mmirs.correct_primary(zv=zvec)
print(m1foc)
t.show_in_notebook()

In [ ]:
new_force_fig = mmirs.telescope.plot_forces(t)
new_force_fig.show()

In [ ]:
zpsf = ZernikeVector()
psf, psf_fig = mmirs.telescope.psf(zpsf, plot=True)
psf_fig.set_size_inches(4.0, 4.0)
psf_fig.show()

# F/9 Dev Section

In [4]:
plt.close('all')
f9wfs = WFSFactory(wfs="newf9")
#f9wfs = WFSFactory(wfs="newf9", pup_offset=[0.0, 0.0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
f9wfs.modes['blue']['reference'].figure.show()

In [ ]:
baseline1_files = ["f9wfs_20180131-191734.fits", "f9wfs_20180131-191925.fits", "f9wfs_20180131-192131.fits"]
baseline2_files = ["f9wfs_20180131-192350.fits", "f9wfs_20180131-192541.fits", "f9wfs_20180131-192731.fits"]
baseline3_files = [
    "f9wfs_20180131-192927.fits", "f9wfs_20180131-193109.fits", 
    "f9wfs_20180131-193325.fits", "f9wfs_20180131-193509.fits"
]

In [ ]:
mask = mk_wfs_mask(results['data'][130:710,60:620], outfile="/Users/tim/MMT/mmtwfs/mmtwfs/data/ref_images/newf9_mask.fits")

In [5]:
plt.close('all')
#f9_file = "/Volumes/LaCie 8TB/wfsdat/20180201/" + baseline3_files[0]
#f9_file = "/Users/tim/MMT/mmtwfs/mmtwfs/data/test_data/test_newf9.fits"
#f9_file = "/Volumes/LaCie 8TB/wfsdat/20180909/f9wfs_20180908-220524.fits"
#f9_file = "/Volumes/LaCie 8TB/wfsdat/20190104/f9wfs_20190104-020657.fits"
#f9_file = "/Volumes/LaCie 8TB/wfsdat/20200218/f9wfs_20200218-044238.fits"
f9_file = "/Volumes/LaCie 8TB/wfsdat/20200226/f9wfs_20200225-205600.fits"
results = f9wfs.measure_slopes(f9_file, 'blue', plot=True)
#plt.scatter(refaps['xcentroid']+results['xcen'], refaps['ycentroid']+results['ycen'])
results['figures']['slopes'].show()
# 265 258
# ref 240 259.5

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'WFS spot detection failed or no spots detected.')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [73]:
zresults = f9wfs.fit_wavefront(results, plot=True)
print(zresults['residual_rms'])
print(zresults['zernike'].pretty_print())
zresults['resid_plot'].show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1106.9186470483617 nm
Fringe Coefficients
 Z02:        -90.52 ±   167 nm 	 X Tilt (1, 1)
 Z03:         272.9 ±   163 nm 	 Y Tilt (1, -1)
 Z04:        -582.6 ±  53.6 nm 	 Defocus (2, 0)
 Z05:        -162.8 ±   109 nm 	 Primary Astig at 45° (2, -2)
 Z06:        -157.9 ±   121 nm 	 Primary Astig at 0° (2, 2)
 Z07:         90.46 ±  72.1 nm 	 Primary Y Coma (3, -1)
 Z08:        -24.81 ±  85.8 nm 	 Primary X Coma (3, 1)
 Z09:        -25.99 ±    97 nm 	 Y Trefoil (3, -3)
 Z10:        -69.38 ±   117 nm 	 X Trefoil (3, 3)
 Z11:         98.87 ±  43.5 nm 	 Primary Spherical (4, 0)
 Z12:        -172.3 ±  79.1 nm 	 Secondary Astigmatism at 0° (4, 2)
 Z13:        -295.3 ±  54.6 nm 	 Secondary Astigmatism at 45° (4, -2)
 Z14:        -58.27 ±   103 nm 	 X Tetrafoil (4, 4)
 Z15:        -75.42 ±  91.9 nm 	 Y Tetrafoil (4, -4)
 Z16:        -302.4 ±  80.4 nm 	 Secondary X Coma (5, 1)
 Z17:        -25.49 ±  57.1 nm 	 Secondary Y Coma (5, -1)
 Z18:         89.11 ±  78.4 nm 	 Secondary X Trefoil (5, 3)
 Z19:

In [74]:
zresults['zernike'].fringe_bar_chart().show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
f9wfs.modes['blue']['reference'].xcen

In [ ]:
refaps = f9wfs.modes['blue']['reference'].masked_apertures
fig, ax = plt.subplots()
fit = results['grid_fit']
refx = refaps['xcentroid'] * (fit['scale'] + refaps['xcentroid'] * fit['xcoma']) + fit['xcen']
refy = refaps['ycentroid'] * (fit['scale'] + refaps['ycentroid'] * fit['ycoma']) + fit['ycen']
x, y = results['apertures'].positions.transpose()[0], results['apertures'].positions.transpose()[1]
spacing = 0.5*(results['xspacing'] + results['yspacing'])
ax.scatter(refx, refy)
ax.set_aspect('equal')
ax.scatter(x, y)
ax.set_xlim(0, 845)
ax.set_ylim(0, 845)
fig.show()
fit['scale']

In [ ]:
data = check_wfsdata(f9_file)
x = np.arange(data.shape[1])
y = np.arange(data.shape[0])
bx = np.arange(data.shape[1]+1)
by = np.arange(data.shape[0]+1)
apertures = results['spots']

# bin the spot positions along the axes and use Lomb-Scargle to measure the grid spacing in each direction
xsum = np.histogram(apertures['xcentroid'], bins=bx)
ysum = np.histogram(apertures['ycentroid'], bins=by)

k = np.linspace(10.0, 50., 500)  # look for spacings from 5 to 50 pixels (plenty of range)
f = 1.0 / k  # convert spacing to frequency
xp = stats.LombScargle(x, xsum[0]).power(f)
yp = stats.LombScargle(y, ysum[0]).power(f)
plt.plot(k, yp)
plt.show()

In [ ]:
zresults['zernike'].bar_chart(last_mode=21).show()

In [ ]:
spots = {'xcentroid': x, 'ycentroid': y}
from scipy import optimize

def match_apertures(refx, refy, spotx, spoty):
    tot_dist = 0.0
    refs = np.array([refx, refy])
    spots = np.array([spotx, spoty])
    match = np.empty(len(refx))
    max_r = spacing / 2.
    for i in np.arange(len(refx)):
        dists = np.sqrt( (spots[0]-refs[0][i])**2 + (spots[1]-refs[1][i])**2 )
        tot_dist += np.min(dists)
    return tot_dist

def fit_apertures(pars, ref, spots):
    xc = pars[0]
    yc = pars[1]
    xscale = pars[2]
    yscale = pars[3]
    xcoma = pars[4]
    ycoma = pars[5]
    refx = ref['xcentroid'] * (xscale + ref['xcentroid'] * xcoma) + xc
    refy = ref['ycentroid'] * (yscale + ref['ycentroid'] * ycoma) + yc
    spotx = spots['xcentroid']
    spoty = spots['ycentroid']
    dist = match_apertures(refx, refy, spotx, spoty)
    return dist

In [ ]:
#%%timeit
args = (refaps, spots)
pars = (results['xcen'], results['ycen'], 1.0, 1.0, 0.0, 0.0)
bounds = (
    (results['xcen']-50, results['xcen']+50), 
    (results['ycen']-50, results['ycen']+50),
    (0.8, 1.2),
    (0.8, 1.2),
    (-0.1, 0.1),
    (-0.1, 0.1)
)
res = optimize.minimize(fit_apertures, pars, args=args, bounds=bounds)
res

In [ ]:
fig, ax = plt.subplots()
refx, refy = refaps['xcentroid'], refaps['ycentroid']
refx = refx * (res['x'][2] + refx*res['x'][4]) + res['x'][0]
refy = refy * (res['x'][3] + refy*res['x'][5]) + res['x'][1]
x, y = results['apertures'].positions.transpose()[0], results['apertures'].positions.transpose()[1]
ax.scatter(refx, refy)
ax.set_aspect('equal')
ax.scatter(x, y)
ax.set_xlim(0, 845)
ax.set_ylim(0, 845)
fig.show()

In [ ]:
print(f9wfs.modes['blue']['reference']['xcen'], f9wfs.modes['blue']['reference']['ycen'])
print(f9wfs.modes['blue']['reference']['xspacing'], f9wfs.modes['blue']['reference']['yspacing'])
f9wfs.modes['blue']['reference'].keys()

In [ ]:
zv = zresults['zernike']
print(zv.rms)
zv.ignore('Z02')
zv.ignore('Z03')
f = zv.plot_map()
f.show()

In [ ]:
bc = zv.bar_chart(residual=zresults['residual_rms'])
bc.show()

In [ ]:
psf, psf_fig = f9wfs.telescope.psf(zv, fov=1.0)
psf_fig.show()

In [ ]:
plt.close('all')

# F/5 Dev Section

In [6]:
f5wfs = WFSFactory(wfs="f5")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
#%%prun
plt.close('all')
#f5_file = "/Volumes/LaCie 8TB/wfsdat/20181022/manual_wfs_0003.fits"
#f5_file = "/Users/tim/MMT/mmtwfs/mmtwfs/data/test_data/auto_wfs_0037_ave.fits"
f5_file = "/Users/tim/MMT/wfsdat/20180923/manual_wfs_0019.fits"
f5_file = "/Volumes/LaCie 8TB/wfsdat/20191015/manual_wfs_0013.fits"
results = f5wfs.measure_slopes(f5_file, 'hecto', plot=True)
results['figures']['slopes'].show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
mask = mk_wfs_mask(results['data'][25:475,25:475], outfile="/Users/tim/MMT/mmtwfs/mmtwfs/data/ref_images/f5_mask.fits")

In [24]:
f5wfs.calculate_recenter(results), results['xcen'], results['ycen']

((<Quantity -3.059 arcsec>, <Quantity -0.797 arcsec>),
 236.00917575744455,
 284.99026226450553)

In [25]:
#%%prun
zresults = f5wfs.fit_wavefront(results, plot=True)
zv = zresults['zernike']
print(zv.pretty_print())
print(zv.rms)
zresults['resid_plot'].show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fringe Coefficients
 Z02:        -57.29 ±   192 nm 	 X Tilt (1, 1)
 Z03:         136.7 ±   186 nm 	 Y Tilt (1, -1)
 Z04:          1117 ±  60.8 nm 	 Defocus (2, 0)
 Z05:         446.6 ±   131 nm 	 Primary Astig at 45° (2, -2)
 Z06:         184.6 ±   131 nm 	 Primary Astig at 0° (2, 2)
 Z07:         14.78 ±  75.3 nm 	 Primary Y Coma (3, -1)
 Z08:         135.8 ±  84.5 nm 	 Primary X Coma (3, 1)
 Z09:         47.62 ±   117 nm 	 Y Trefoil (3, -3)
 Z10:         128.1 ±   108 nm 	 X Trefoil (3, 3)
 Z11:        -63.59 ±  49.1 nm 	 Primary Spherical (4, 0)
 Z12:        -94.48 ±  64.1 nm 	 Secondary Astigmatism at 0° (4, 2)
 Z13:        -130.8 ±  55.2 nm 	 Secondary Astigmatism at 45° (4, -2)
 Z14:         16.49 ±  93.6 nm 	 X Tetrafoil (4, 4)
 Z15:        -2.547 ±  85.9 nm 	 Y Tetrafoil (4, -4)
 Z16:        -132.9 ±  61.3 nm 	 Secondary X Coma (5, 1)
 Z17:        -17.24 ±  59.3 nm 	 Secondary Y Coma (5, -1)
 Z18:         -1.69 ±  55.4 nm 	 Secondary X Trefoil (5, 3)
 Z19:        -3.584 ±  61.9

In [26]:
zf = zv.fringe_bar_chart()
zf.show()
zresults['residual_rms']

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Quantity 1363.48022001 nm>

In [ ]:
bc = zv.bar_chart(last_mode=21)
bc.show()

In [ ]:
zv.denormalize()
zv

In [ ]:
refaps = f5wfs.modes['hecto']['reference'].masked_apertures
fig, ax = plt.subplots()
fit = results['grid_fit']
refx = refaps['xcentroid'] * (fit['scale'] + refaps['xcentroid'] * fit['xcoma']) + fit['xcen']
refy = refaps['ycentroid'] * (fit['scale'] + refaps['ycentroid'] * fit['ycoma']) + fit['ycen']
x, y = results['apertures'].positions.transpose()[0], results['apertures'].positions.transpose()[1]
spacing = 0.5*(results['xspacing'] + results['yspacing'])
ax.scatter(refx, refy)
ax.set_aspect('equal')
ax.scatter(x, y)
ax.set_xlim(0, 512)
ax.set_ylim(0, 512)
fig.show()

In [ ]:
zv.ignore('Z02')
zv.ignore('Z03')
zv.ignore('Z04')
print(zv.rms)
mf = zv.plot_map()
mf.show()

In [ ]:
t, m1foc = f5wfs.telescope.calculate_primary_corrections(zresults['zernike'])

In [ ]:
fplot = f5wfs.telescope.plot_forces(t, m1focus=m1foc)
fplot.show()

In [ ]:
t.show_in_notebook()

# Old F/9

In [ ]:
oldf9 = WFSFactory(wfs="f9")

In [ ]:
#%%prun
plt.close('all')
oldf9_file = "/Users/tim/MMT/mmtwfs/mmtwfs/data/test_data/TREX_p500_0000.fits"
oldf9_file = "/Volumes/LaCie 8TB/wfsdat/20031223/start_0001.fits"
results = oldf9.measure_slopes(oldf9_file, 'blue', plot=True)
results['figures']['slopes'].show()

In [ ]:
mask = mk_wfs_mask(results['data'][40:460,45:465], outfile="/Users/tim/MMT/mmtwfs/mmtwfs/data/ref_images/oldf9_mask.fits")

In [ ]:
oldf9.ref_spot_fwhm()

# Scratch space

In [ ]:
cen = center_pupil(subt, pup, plot=True)
print(cen)
plt.show()

In [ ]:
from astropy.io import ascii

In [ ]:
t = f9wfs.telescope
m = t.zern_map

In [ ]:
z = np.loadtxt("/Users/tim/MMT/wfsdat/20170110/mmirs_wfs_0375.dao.sub.zrn.cor")
z1 = np.loadtxt("/Users/tim/MMT/wfsdat/20170110/mmirs_wfs_0375.dao.av.zrn")

In [ ]:
sigma = results['fwhm'] * stats.funcs.gaussian_fwhm_to_sigma

wave = 650 * u.nm
wave = wave.to(u.m).value  # r_0 equation expects meters so convert
# calculate the physical size of each aperture.
ref = mmirs.modes['mmirs2']['reference']
apsize_pix = np.max((ref['xspacing'], ref['yspacing']))
d = mmirs.telescope.diameter * apsize_pix / mmirs.pup_size
d = d.to(u.m).value  # r_0 equation expects meters so convert

# we need to deconvolve the instrumental spot width from the measured one
ref_sigma = ref['sigma']
if sigma > ref_sigma:
    corr_sigma = np.sqrt(sigma**2 - ref_sigma**2)
else:
    corr_sigma = 0.0
corr_sigma *= mmirs.pix_size.to(u.rad).value  # r_0 equation expects radians so convert
# this equation relates the motion within a single aperture to the characteristic scale size of the
# turbulence, r_0.
r_0 = ( 0.179 * (wave**2) * (d**(-1./3.))/corr_sigma**2 )**0.6
print(r_0)
# this equation relates the turbulence scale size to an expected image FWHM at the given wavelength.
raw_seeing = u.rad * 0.98 * wave / r_0
raw_seeing = raw_seeing.to(u.arcsec)


In [ ]:
t = bino.telescope

In [ ]:
z = ZernikeVector(Z11=-1000*u.nm)
z.plot_map()
plt.show()

In [ ]:
ft, m1, zv_masked = t.calculate_primary_corrections(z)

In [ ]:
ft['force'].min()

In [ ]:
m1

In [ ]:
derot_phi = phi - u.Quantity(225*u.deg, u.rad).value
pol2cart([dr, derot_phi])

In [ ]:
u.Quantity(225*u.deg, u.rad).value

In [ ]:
dx, dy

In [ ]:
t = mmirs.telescope
zv = ZernikeVector(Z05=1000, Z11=250)
force, focus = t.correct_primary(zv)
f1 = focus.copy()
uforce, ufocus = t.undo_last()
print(ufocus)
print(f1)
assert(ufocus == -1 * f1)

In [ ]:
focus.copy()

In [ ]:
np.allclose(np.zeros(2), 0.0)

In [ ]:
s = "offset_inc wfs z 200.3"
assert("200.3" in s)

In [ ]:
fig, ax = plt.subplots()

In [ ]:
ax.imshow(results['data'], cmap=cm.Greys)
fig.savefig("test.png")

In [ ]:
a = fig.axes[0]

In [ ]:
fig.show()

In [ ]:
"{0:0.0f}".format((550 * u.nm))

In [ ]:
x, y, r, p, ph = zvec.phase_map(n=100)
i = np.argsort(x)

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [ ]:
data = [go.Heatmap(z=np.array(ph[i]), x=x[i], y=y[i], colorscale='RdBu')]
layout = go.Layout(title="test")
fig = go.Figure(data=data, layout=layout)
plot(fig, filename="blah")

In [ ]:
t = MMT()
zv = ZernikeVector(Z05=1000, Z11=250)
force, focus = t.calculate_primary_corrections(zv)

In [ ]:
lforce, lfocus = t.correct_primary(force, focus)

In [ ]:
uforce, ufocus = t.undo_last()

In [ ]:
newf9 = WFSFactory(wfs="newf9")

In [ ]:
newf9.modes['blue']['reference']

In [ ]:
x = newf9.modes['blue']['reference']['apertures']['xcentroid'] + newf9.modes['blue']['reference']['xcen']

In [ ]:
y = newf9.modes['blue']['reference']['apertures']['ycentroid'] + newf9.modes['blue']['reference']['ycen']

In [ ]:
arr = np.array((x.data, y.data)).transpose()

In [ ]:
np.savetxt("blah.txt", arr, fmt="%.3f")

In [ ]:
x, y = 405, 445
xnew, ynew = 438, 469
xref, yref = 433, 431
pixsize = 0.09639

rot = u.Quantity(-225 * u.deg, u.rad).value

dx = x - xref
dy = y - yref

dr, phi = cart2pol([dx, dy])

derot_phi = phi - rot

az, el = pol2cart([dr, derot_phi])
print(az, el)

In [ ]:
x, y = 405, 445
xnew, ynew = 438, 469
xref, yref = 433, 431
pixsize = 0.09639

rot = u.Quantity(-225 * u.deg, u.rad).value

dx = 1 * (x - xref)
dy = 1 * (y - yref)

dr, phi = cart2pol([dx, dy])

derot_phi = phi - rot

az, el = pol2cart([dr, derot_phi])
print(az, el)

In [ ]:
azcorr = 3.0 / pixsize
elcorr = 0.9 / pixsize
drc, phic = cart2pol([azcorr, elcorr])
derot_phic = phic + rot
az_pix, el_pix = pol2cart([drc, derot_phic])
print(az_pix, el_pix)

In [ ]:
a = np.arange(5)

In [ ]:
np.delete(a, 2)

In [ ]:
2320-2820

In [ ]:
x1, y1 = 274, 288
x2, y2 = 243, 314
xc, yc = 255, 255
rot_def = 234 * u.deg
rotation = u.Quantity(rot_def, u.rad).value

In [ ]:
x1, y1 = 212, 238
x2, y2 = 243, 314
xc, yc = 255, 255
rot_def = 234 * u.deg
rotation = u.Quantity(rot_def, u.rad).value

In [ ]:
x1, y1 = 358, 308
xc, yc = 419, 439
rot_def = -225 * u.deg
rotation = u.Quantity(rot_def, u.rad).value

In [ ]:
dx = x1 - xc
dy = y1 - yc

In [ ]:
dr, phi = cart2pol([dx, dy])

In [ ]:
derot_phi = rotation + phi

In [ ]:
az, el = pol2cart([dr, derot_phi])
az, el

In [ ]:
data = check_wfsdata("/Users/tim/MMT/wfsdat/20180604/wfs_ff_cal_img_2018.0604.052401.fits")
#data = check_wfsdata("/Users/tim/MMT/wfsdat/20171101/mmirs_wfs_0022.fits")
#data = check_wfsdata("/Volumes/LACIE SHARE/wfsdat/20180424/f9wfs_20180423-234645.fits")
#data = check_wfsdata("/Users/tim/MMT/wfsdat/20180404/manual_wfs_0004.fits")
fix, ax = plt.subplots()
norm = wfs_norm(data)
ax.imshow(data, cmap='Greys', origin='lower', norm=norm, interpolation='None')
ax.set_title("Binospec WFS")
plt.savefig("/Users/tim/MMT/spie/2018/bino.pdf")
plt.show()

In [ ]:
refdata = check_wfsdata("/Users/tim/MMT/mmtwfs/mmtwfs/data/ref_images/f9_new_ref.fits")
refdata = refdata - np.median(refdata)
apertures, figure = wfsfind(refdata, fwhm=4.0, threshold=30, plot=True)
figure.show()

In [16]:
0.8 * .328e9 * 0.002

524800.0

In [17]:
blah = (
    "this"
    "is"
    "a"
    "string"
)

In [18]:
blah

'thisisastring'